In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
def check_null_count(df):
    null_sum = df.isnull().sum()
    return len(null_sum[null_sum!=0])

check_null_count(df)

In [ ]:
def check_uniform_col(df):
    for col in df.columns:
        if df[col].nunique()==1:
            print(col)

check_uniform_col(df)

In [ ]:
df_new = df.drop(['X_04', 'X_23', 'X_47', 'X_48'], axis=1)
df_new.head()

In [ ]:
def check_multicolinear(df):
    
    df_corr = df.corr()
    rows, cols = np.where(df_corr==1)
    row_names = df_corr.index[rows]
    col_names = df_corr.columns[cols]
    
    for row, col in zip(row_names, col_names):
        if row != col:
            print(f"행: {row}, 열: {col}") 

check_multicolinear(df_new)

In [ ]:
x_feature = df.filter(regex='^X_*') # Input : X Featrue
y_feature = df.filter(regex='^Y_*') # Output : Y Feature

In [ ]:
from sklearn.model_selection import train_test_split

x_train_val, x_test, y_train_val, y_test = train_test_split(x_feature, y_feature, test_size=0.2, random_state=0)
x_train, x_val, y_train, y_val =train_test_split(x_train_val, y_train_val, test_size = 0.25, random_state=0)

print(f'x_train: {x_train.shape}, x_val: {x_val.shape}, x_test: {x_test.shape}')
print(f'y_train: {x_train.shape}, y_val: {x_val.shape}, y_test: {y_test.shape}')

In [ ]:
X.describe()

In [ ]:
X.info()

In [ ]:
Y.head()

In [ ]:
Y.shape

In [ ]:
Y.describe()

In [ ]:
Y.info()

In [ ]:
X.isnull

In [ ]:
Y.isnull

In [ ]:
#중간 검사 결과 모두 통과 -> X_04, X_23, X_47, X_48 제외 

X_mid = X.loc[:,['X_04','X_23', 'X_47', 'X_48']]
X_mid.describe()

In [ ]:
#viz function

#x column's distplot
def show_dist_plot_x(columns,data):
    i = 0
    plt.figure(figsize=(24,16))
    plt.subplots(8,7,figsize=(24,16))
    for col in columns:
        i += 1
        plt.subplot(8,7,i)
        sns.distplot(data[col])
    plt.tight_layout()

#y column's distplot
def show_dist_plot_y(columns,data):
    i = 0
    plt.figure(figsize=(18,8))
    plt.subplots(2,7,figsize=(18,8))
    for col in columns:
        i += 1
        plt.subplot(2,7,i)
        sns.distplot(data[col],color='skyblue')
    plt.tight_layout()
    
#violinplot
def show_dist_plot_vio(columns,data):
    i = 0
    plt.figure(figsize=(24,16))
    plt.subplots(8,7,figsize=(24,16))
    for col in columns:
        i += 1
        plt.subplot(8,7,i)
        sns.boxplot(data[col],color='skyblue')
    plt.tight_layout()

#plot
def show_plot(data,columns):
    for col in columns:
        plt.figure(figsize=(36,12))
        plt.plot(data[col])
        plt.title(f'{col}', fontsize=40)
        plt.tight_layout()

In [ ]:
sns.set_style('whitegrid')
show_dist_plot_x(df.filter(regex='X').columns, df)

In [ ]:
show_dist_plot_vio(df.filter(regex='X').columns, df)

In [ ]:
show_dist_plot_y(df.filter(regex='Y').columns, df)

In [ ]:
show_dist_plot_vio(df.filter(regex='Y').columns, df)

In [ ]:
#단계별 누름량(X01, X02, X05, X06)

show_plot(X,["X_01", "X_02", "X_05", "X_06"])

In [ ]:
#방열 재료 무게(X03, X10, X11)

show_plot(X,["X_03", "X_10", "X_11"])

#방열재료 무게1(X03)에서 변곡 확인 
#방열재료 무게2,3 (X10,X11)의 대부분 무게가 0 -> 의미가 있는 데이터인지 확인 필요

In [ ]:
#X10, X11의 결측치 확인
zero_count_X10 = (X['X_10'] == 0).sum()
zero_count_X11 = (X['X_11'] == 0).sum()
print(zero_count_X10, zero_count_X11)

In [ ]:
show_plot(X,["X_03", "X_10", "X_11"])
show_plot(Y,['Y_01', 'Y_02', 'Y_03'])

In [ ]:
#방열 재료 면적(X07, X08, X09)

show_plot(X,["X_07", "X_08", "X_09"])

#이상치가 많이 존재함 

In [ ]:
show_plot(X,["X_07", "X_08", "X_09"])
show_plot(Y,['Y_01', 'Y_02', 'Y_03'])

In [ ]:
#상관성 확인 -> 나중에 다시 하기 
X_corr = X.corr() #Pearson 상관 계수 
Y_corr = Y.corr()

In [ ]:
X_corr_new = X_corr.apply(lambda x : round(x,1)) #X Feauter 중 이진 data 제거 


In [ ]:
plt.figure(figsize=(22,14))
sns.heatmap(X_corr_new, cmap='RdBu_r', vmax=1, vmin=-1, annot=True, fmt='.1f')

In [ ]:
plt.figure(figsize=(22,14))
sns.heatmap(Y_corr, cmap='RdBu_r', vmax=1, vmin=-1, annot=True, fmt='.1f') #Threshold를 몇으로 하는게 적절한지??? 